In [ ]:
import boto3
import json
import pprint
import os
from dotenv import load_dotenv
load_dotenv()

pp = pprint.PrettyPrinter(indent=2)

# login for boto3 credentials
boto3.setup_default_session(profile_name=os.environ['AWS_PROFILE_NAME'])
client = boto3.client('bedrock')
bedrock_runtime_client = boto3.client('bedrock-runtime')

boto3.client('sts').get_caller_identity()

In [ ]:
# models details
response = client.list_foundation_models()
models = response['modelSummaries']
models_details = [f"{model['providerName']}, {model['modelName']}, {model['modelId']}, {model['modelArn']}, inputModalities: {','.join(model['inputModalities'])}, outputModalities: {','.join(model['outputModalities'])}" for model in models]
models_details.sort()
print('\n'.join(models_details))

In [ ]:
# text prompt -> text generation
prompt  = 'list three names of people'
body = json.dumps(
                {
                    "prompt": prompt,
                    "maxTokens": 500,
                    "temperature": 0,
                    "stopSequences": [],
                })
response = bedrock_runtime_client.invoke_model(modelId='ai21.j2-ultra', body=body)
response_body = json.loads(response.get("body").read())
response_body

In [ ]:
# stream response
# Define prompt and model parameters
prompt_data = """Write an essay about why someone should drink coffee"""

text_gen_config = {
    "maxTokenCount": 1000,
    "stopSequences": [], 
    "temperature": 0,
    "topP": 0.9
}

body = json.dumps({
    "inputText": prompt_data,
    "textGenerationConfig": text_gen_config  
})

model_id = 'amazon.titan-tg1-large'
accept = 'application/json' 
content_type = 'application/json'

#invoke the model with a streamed response 
response = bedrock_runtime_client.invoke_model_with_response_stream(
    body=body, 
    modelId=model_id, 
    accept=accept, 
    contentType=content_type
)

for event in response['body']:
    data = json.loads(event['chunk']['bytes'])
    print(data['outputText'])

In [ ]:
# titan embedding - embed `prompt` and print

# Define prompt and model parameters
prompt_data = """Write me a poem about apples"""

body = json.dumps({
    "inputText": prompt_data,
})

model_id = 'amazon.titan-embed-g1-text-02' #look for embeddings in the modelID
accept = 'application/json' 
content_type = 'application/json'

# Invoke model 
response = bedrock_runtime_client.invoke_model(
    body=body, 
    modelId=model_id, 
    accept=accept, 
    contentType=content_type
)

# Print response
response_body = json.loads(response['body'].read())
embedding = response_body.get('embedding')

#Print the Embedding

print(embedding)

In [ ]:
# langchain bedrock integration
from langchain_community.llms import Bedrock
import os

llm = Bedrock(
    credentials_profile_name=os.environ['AWS_PROFILE_NAME'], model_id="amazon.titan-text-express-v1"
)

llm.invoke('who are you?')

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation = ConversationChain(
    llm=llm, verbose=True, memory=ConversationBufferMemory()
)

conversation.predict(input="what is the weather in philly")
conversation.predict(input="what is the population there?")
conversation.predict(input="who are famous actors from there?")
conversation.predict(input="what is the the most famous food there?")